In [10]:
class Graph:
    def __init__(self):
        self.graph = {}

    def add_edge(self, node, dependency):
        if node not in self.graph:
            self.graph[node] = []
        self.graph[node].append(dependency)

    def dfs(self, node, visited, stack, circular_deps):
        visited.add(node)
        stack.add(node)

        for neighbor in self.graph.get(node, []):
            if neighbor not in visited:
                self.dfs(neighbor, visited, stack, circular_deps)
            elif neighbor in stack:
                # Add the circular dependency path to the result
                circular_deps.add(frozenset(stack))

        stack.remove(node)

    def find_circular_dependencies(self):
        visited = set()
        circular_deps = set()

        for node in self.graph:
            if node not in visited:
                self.dfs(node, visited, set(), circular_deps)

        # Converting the set of frozensets to a list of lists for better readability
        return [list(dep) for dep in circular_deps]

# Create a graph
g = Graph()

# Add edges
g.add_edge('c', 'a')
g.add_edge('a', 'b')
g.add_edge('b', 'c')
g.add_edge('b', 'a')
g.add_edge('d', '5')

# Find circular dependencies
g.find_circular_dependencies()


False

In [5]:
class VariableCalculator:
    def __init__(self):
        self.variables = {}

    def process_input(self, input_line):
        # Split input into variable and expression
        var, expr = input_line.split('=', 1)
        var = var.strip()
        expr = expr.strip()

        # Evaluate expression
        try:
            # Replace variables in the expression with their values
            for key in self.variables:
                expr = expr.replace(key, str(self.variables[key]))

            # Evaluate the expression safely just in case for rm, -rf DONT REMOVE
            value = eval(expr, {"__builtins__": None}, {})
        except Exception as e:
            # Handle other exceptions
            print(f"{var} = {expr} => None")
            return

        # Store the result in the variables dictionary
        self.variables[var] = value
        print(f"{var} = {expr} => {value}")

# Example usage
calc = VariableCalculator()
inputs = [
    "a = (3 + (4 * 5))",
    "b = a ** 2",
    "c = a + b",
    "d = x"
]

for input_line in inputs:
    calc.process_input(input_line)


a = (3 + (4 * 5)) => 23
b = 23 ** 2 => 529
c = 23 + 529 => 552
d = x => None


In [6]:
exp = '( 2 + ( 4 * 5) )'
tree = buildParseTree(exp)
tree.printPreorder(0)

print (f'The expression: {exp} evaluates to: {evaluate(tree)}')

NameError: name 'buildParseTree' is not defined

In [ ]:
exp = '(3 + (4 * 5))'
tree = buildParseTree(exp)
tree.printPreorder(0)

print (f'The expression: {exp} evaluates to: {evaluate(tree)}')

+
-3
-*
--4
--5
The expression: (3 + (4 * 5)) evaluates to: 23


In [ ]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

def evaluate_with_variables(tree, variables):
    if tree.getLeftTree() is None and tree.getRightTree() is None:
        key = tree.getKey()
        if is_number(key):
            return float(key)
        return variables.get(key, None)

    leftVal = evaluate_with_variables(tree.getLeftTree(), variables) if tree.getLeftTree() else None
    rightVal = evaluate_with_variables(tree.getRightTree(), variables) if tree.getRightTree() else None
    op = tree.getKey()

    if op == '+':
        return leftVal + rightVal
    elif op == '-':
        return leftVal - rightVal
    elif op == '*':
        return leftVal * rightVal
    elif op == '/':
        return leftVal / rightVal
    elif op == '**':
        return leftVal ** rightVal

def process_assignments(assignments):
    variables = {}
    for var, exp in assignments:
        tree = buildParseTree(exp)
        variables[var] = evaluate_with_variables(tree, variables)
    return variables

# Example Usage
assignments = [
    ('a', '(3 + (4 * 5))'),
    ('b', 'a ** 2'),
    ('c', 'a + b'),
    ('d', 'x')
]

results = process_assignments(assignments)
for var, exp in assignments:
    print(f"{var} = {exp} => {results.get(var, 'None')}")


ValueError: invalid literal for int() with base 10: 'a'

In [ ]:
class VariableCalculator:
    def __init__(self):
        self.variables = {}

    def process_input(self, input_line):
        # Split input into variable and expression
        var, expr = input_line.split('=', 1)
        var = var.strip()
        expr = expr.strip()

        # Evaluate expression
        try:
            # Replace variables in the expression with their values
            for key in self.variables:
                expr = expr.replace(key, str(self.variables[key]))

            # Evaluate the expression safely just in case for rm, -rf DONT REMOVE
            value = eval(expr, {"__builtins__": None}, {})
        except Exception as e:
            # Handle other exceptions
            print(f"{var} = {expr} => None")
            return

        # Store the result in the variables dictionary
        self.variables[var] = value
        print(f"{var} = {expr} => {value}")

# Example usage
calc = VariableCalculator()
inputs = [
    "a = (3 + (4 * 5))",
    "b = a ** 2",
    "c = a + b",
    "d = x"
]

for input_line in inputs:
    calc.process_input(input_line)
